In [2]:
#Libraries: Numpy, Matplotlib, Random, Scipy

import numpy as np
import matplotlib.pyplot as plt
import random
from scipy.interpolate import lagrange 

In [3]:
#A) PARAMETROS GENERALES

#Grados de libertad de la estrcutura
grado = 4

#Matriz de rigidez estructura sin daños
rigidez_SIN_daño = (np.array([[693,-231,0,0,0], [-231,462,-231,0,0], [0,-231,462,-231,0],[0,0,-231,462,-231],[0,0,0,-231,693]]))*100000

#Matriz de masas
Matriz_Masas = np.array([[1980,0,0,0,0],[0,1980,0,0,0], [0,0,1980,0,0], [0,0,0,1980,0],[0,0,0,0,1980]])

#B) FUNCION PARA DETERMINAR FRECUENCIAS NATURALES Y MODOS DE VIBRACION

def FreqYMod(b, h, E, eventos):
    
    #1) MATRICES USADAS EN EL CODIGO
    MMatriz_total = []
    fr_CON_daños = []
    mod_CON_daños = []
    ultimo = []
    ante = []
    iterador = []
    matriz = []
    I = []
    
    #2) MOMENTO DE INERCIA DE LA SECCION
    inercia = b*(h**3)*(1/12)
    
    #3) GENERACION DE UN VECTOR DE RIGIDEZ FLEXIONAL EI EN CADA BARRA 
    
    #matriz de porcentajes de inercia en cada barra representando la exixstencia de un daño
    for j in range(0,eventos):
        daño = [random.randint(0, 5), random.randint(30, 95)] #generacion un daño aleatorio en una barra aleatoria
        
        z = [100,100,100,100,100,100]
        for i in range(0,6):
            if daño[0] == i:
                z.remove(z[i])
                z.insert(i,daño[1])
                I.append(z)
                
    #matriz de rigidez flexional en cada barra teniendo en cuenta el daño en la barra
    for row in I:
        iterador = []
        for elem in row:
            v = elem*(inercia*E/100)
            iterador.append(v)
        matriz.append(iterador)
    
    #4) CONFECCION DE LA MATRIZ DE RIGIDEZ DE LA ESTRUCTURA

    L1 = 8 #longitud tramo 1
    L2 = 64 #longitud tramos 2,3,4,5
    L3 = 8 #longitud tramo 6

    #generacion de la matriz de rigidez para cada evento de daños
    for row in matriz:
        r11 = 3*row[0]/L1 + 12*row[1]/L2
        r12 = -12*row[1]/L2
        r13 = 0
        r14 = 0
        r15 = 0

        r21 = -12*row[1]/L2
        r22 = 12*row[1]/L2 + 12*row[2]/L2
        r23  =-12*row[2]/L2
        r24 = 0
        r25 = 0

        r31 = 0
        r32 = -12*row[2]/L2
        r33 = 12*row[2]/L2 + 12*row[3]/L2
        r34 = -12*row[3]/L2
        r35 = 0

        r41 = 0
        r42 = 0
        r43 = -12*row[3]/L2
        r44 = 12*row[3]/L2 + 12*row[4]/L2
        r45 = -12*row[4]/L2

        r51 = 0
        r52 = 0
        r53 = 0
        r54 = -12*row[4]/L2
        r55 = 12*row[4]/L2 + 12*row[5]/L3

        kk = np.array([[r11,r12,r13,r14,r15],[r21,r22,r23,r24,r25],[r31,r32,r33,r34,r35],[r41,r42,r43,r44,r45],[r51,r52,r53,r54,r55]])

        MMatriz_total.append(kk) #matriz de matrices de rigidez
            
    #5) CALCULO DE AUTOVALORES (FRECUENCIAS NATURALES) Y AUTOVECTORES (MODOS DE VIBRACION)
    
    for K in MMatriz_total:
        #Calculo los autovalores y autovectores de la matriz de rigidez K usando numpy
        autovalores, autovectores = np.linalg.eig(K)

        #Obtencion de las frecuencias naturales sabiendo que K*autovectores = autovalores*autovectores
        #Siendo: autovector = matriz modal; autovalor = landa; siendo landa=frecuencia^2*M --> despejo la frecuencia aplicando matriz inversa
        inversa_M = np.linalg.inv(Matriz_Masas) #calculo la matriz inversa de M (matriz de masa)
        frecuencia_matriz = (autovalores*inversa_M)**(1/2) #despejo el vector de frecuencias naturales al cuadrado

        #Represtancion de la frecuencia natural como vector y en orden
        vector_frecuencia = []
        for row in frecuencia_matriz:
            for element in row:
                if element !=0:
                    vector_frecuencia.append(element)

        frecuencia = np.around(sorted(vector_frecuencia),2)

        #Ordenamiento de la matriz de autovectores para que queden las formas modales en orden (modo1..modo2..modo3..modo4..modoN)
        autovec = np.around(autovectores,2)
        autovectores2=[]
        count = grado+1

        for i in autovec.transpose():
            count-=1
            autovectores2.append(autovec.transpose()[count])

        c = np.around(autovectores2,2)

        #Calculo del maximo de la matriz para obtener desplazamientos respecto a un valor unitario
        maximos = []

        for i in abs(c):
            maximo = np.amax(i)
            maximos.append(maximo)


        #Division de la matriz c (formas modales ordenadas numericamente) por los valores maximos de cada autovector y obtencion las formas modales    
        autovectores3=[]
        j = 0
        for i in c:
            z = i/maximos[j]
            autovectores3.append(z)
            j+=1

        modos = np.around(autovectores3,3)
        c = frecuencia + modos
        fr_CON_daños.append(frecuencia)
        mod_CON_daños.append(modos)
        
        #6) REPRESENTACION GRAFICA DE LOS MODOS DE VIBRACION DE LAS ESTRUCTURAS GENERADAS (igual al numero de eventos de daños generados)
        
        """SI SE QUIERE GRAFICAR CADA UNO DE LOS MODOS HABILITO ESTE CODIGO - NO SE REALIZA YA QUE IMPLICARIA LA CONFECCION DE MILES DE GRAFICOS SIN SENTIDO
        for i in range(0,len(modos)):   
            viga = np.array([2,6,10,14,18])
            plt.figure(f'plot{i}')    # Crea una ventana titulada 'plot'
            plt.figure(f'plot{i}') # Ahora cambiamos a la ventana 'plot'
            plt.figure(figsize=(5,5))

            #plt.plot(viga,modos[i],"g") #dibujo los puntos unidos por lineas (modo de vibracion)
            p = lagrange(viga,modos[i])
            xp = np.linspace(2,18,100)
            yp = p(xp)
            plt.plot(xp,yp, ls="--",color =  "r")


            viga0 = np.array([0,2])
            modo0 = np.array([0,modos[i][0]])

            j = lagrange(viga0,modo0)
            xj = np.linspace(0,2,100)
            yj = j(xj)
            plt.plot(xj,yj, ls="--", color = "r")



            #plt.plot(viga0,modo0,"g")

            vigaf = np.array([20,18])
            modof = np.array([0,modos[i][n_matriz]])

            h = lagrange(vigaf,modof)
            xh = np.linspace(18,20,100)
            yh = h(xh)
            plt.plot(xh,yh,ls="--",color =  "r")

            barr = np.array([0,20])
            level = np.array([0,0])
            plt.plot(barr,level, color = "black")


            #plt.plot(vigaf,modof,"g")
            plt.title(f"Modo {i+1}")"""
    
    #7) GENERACION DE LA MATRIZ QUE CONTIENE TODOS LOS EVENTOS DE DAÑO
    #Esta matriz contiene para cada estructura con daño: vector de frecuencias naturales, vector modo de vibracion, vector % de rigidez flexion, vector % de inercias
    
    contador = 0   
    last = [] #Matriz con el conjunto de datos requerido como input en la red neuronal
    for j in range(0,len(I)):
        contador+=1
        ultimo = []
        ultimo.append(contador)
        for i in range(0,5):
            ultimo.append(fr_CON_daños[j][i])
        for i in range(0,5):
             ultimo.append(mod_CON_daños[j][4][i])
        for i in range(0,6):
             ultimo.append(matriz[j][i])
        for i in range(0,6):
            ultimo.append(I[j][i])
        last.append(ultimo)
        
    #8) GUARDADO DEL CONJUNTO DE DATOS GENERADO EN UN ARCHIVO .CSV    
        
    np.savetxt("input_pasarela5.csv", last, delimiter=",")  
        
    return last
        


# EJECUCION DEL CODIGO 
# b = 11m, h = 1.8m, E = 23500000 kN/m2, cantidad de eventos de daño generados = 15000

FreqYMod(11, 1.8, 23500000, 40000) 
    
       #INPUTS DE LA FUNCION (4 parametros 1x1)
#b: base de la seccion en metros
#h: altura de la seccion en metros
#E: modulo de elasticidad del material en kN/m2
#eventos: cantidad de eventos de daño aleatorios que se quieren generar
   
    #OUTPUTs DE LA FUNCION: 

#fr_CON_daños: frecuencias naturales en Hz de los 5 modos de cada evento de daño generado (vector 1x5)
#mod_CON_daños: modo 1 de vibracion de cada evento de daño generado (vector 1x5)
#matriz: rigidez flexional EI de cada una de las 6 barras de cada evento de daño generado
#I: reduccion de la inercia de cada barra segun daño generado (vector 1x6)
    
    

[[1,
  72.39,
  125.27,
  181.79,
  206.95,
  329.8,
  0.313,
  0.791,
  1.0,
  0.701,
  0.075,
  125631000.00000003,
  125631000.00000003,
  80403840.00000001,
  125631000.00000003,
  125631000.00000003,
  125631000.00000003,
  100,
  100,
  64,
  100,
  100,
  100],
 [2,
  61.8,
  128.82,
  183.79,
  214.2,
  317.4,
  -0.286,
  -0.762,
  -1.0,
  -0.921,
  -0.048,
  125631000.00000003,
  125631000.00000003,
  125631000.00000003,
  125631000.00000003,
  55277640.00000001,
  125631000.00000003,
  100,
  100,
  100,
  100,
  44,
  100],
 [3,
  67.42,
  136.94,
  179.15,
  200.57,
  329.81,
  -0.25,
  -0.953,
  -1.0,
  -0.672,
  -0.078,
  125631000.00000003,
  74122290.00000001,
  125631000.00000003,
  125631000.00000003,
  125631000.00000003,
  125631000.00000003,
  100,
  59,
  100,
  100,
  100,
  100],
 [4,
  69.77,
  134.73,
  185.99,
  214.44,
  325.25,
  -0.323,
  -0.831,
  -1.0,
  -0.754,
  -0.077,
  125631000.00000003,
  125631000.00000003,
  125631000.00000003,
  125631000.00000